In [3]:
import pandas as pd


In [10]:
train_label = pd.read_csv('../data/FairFace/fairface_label_train.csv')
val_label = pd.read_csv('../data/FairFace/fairface_label_val.csv')
val_label.age.isin(train_label.age.unique()).sum()

10954

In [11]:
assert val_label.age.isin(train_label.age.unique()).sum() == val_label.shape[0]
assert train_label.age.isin(val_label.age.unique()).sum() == train_label.shape[0]


In [15]:
train_label.age.unique()

array(['50-59', '30-39', '3-9', '20-29', '40-49', '10-19', '60-69', '0-2',
       'more than 70'], dtype=object)

In [16]:
AGE_BUCKETS = ('0-2', '3-9', '10-19', '20-29', '30-39' , '40-49', '50-59', '60-69', 'more than 70')

In [17]:
train_label.age.apply(lambda x: AGE_BUCKETS.index(x)).value_counts()

3    25598
4    19250
5    10744
1    10408
2     9103
6     6228
7     2779
0     1792
8      842
Name: age, dtype: int64

In [18]:
train_label

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
86739,train/86740.jpg,20-29,Male,Indian,True
86740,train/86741.jpg,10-19,Male,Indian,True
86741,train/86742.jpg,more than 70,Female,Indian,True
86742,train/86743.jpg,10-19,Female,Black,True


In [20]:
import albumentations as A

In [21]:
import timm

In [27]:
m = timm.create_model("resnet18", features_only=True)

In [34]:
ch=m.feature_info.channels()[-1]

In [35]:
import torch 
import torch.nn as nn
from coral_pytorch.layers import CoralLayer
output_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            CoralLayer(size_in=ch,
                                  num_classes=9)
        )
m(torch.randn(1, 3, 224, 224))[-1].shape

torch.Size([1, 512, 7, 7])

In [36]:
output_layer(m(torch.randn(1, 3, 224, 224))[-1])

tensor([[0.9525, 0.8275, 0.7025, 0.5775, 0.4525, 0.3275, 0.2025, 0.0775]],
       grad_fn=<AddBackward0>)